## TrOCR Fine Tune


##### Install Necessary Libraries

In [ ]:
# %pip install -q torch torchvision torchaudio
# %pip install -q datasets jiwer

##### Import Necessary Libraries

In [ ]:
import os, sys, itertools
os.environ['TOKENIZERS_PARALLELISM']='false'
# export CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

import cv2
from PIL import Image

import torch
from torch.utils.data import Dataset

import datasets
from datasets import load_dataset

import transformers
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, default_data_collator
from datasets import load_metric


from preprocess_image import PreprocessImage
from augment_image import AugmentImage

##### Ingest, Preprocess, & Split Dataset (into Training & Testing Datasets)

In [ ]:
df_train = pd.read_csv('train/DataTrain.csv', delimiter=';')
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_train = df_train.drop([126, 457, 600]) # delete the wrong labeled data
df_train.head()

##### Show First Samples in DataFrame

In [ ]:
df_train.head(12)

##### Prepare the image and label dataset

###### Prepare Label & Images

In [ ]:
label_array = df_train['Vehicleregistrationplate'].to_numpy()
path_train = df_train['NameofFile'].to_numpy()

###### Train val split

In [ ]:
train_path_array, test_path_array, train_label_array, test_label_array = train_test_split(path_train, label_array, test_size=0.2, random_state=42)

###### Preprocess Test Images

In [ ]:
test_image = PreprocessImage(img_paths=test_path_array, output_pixel=384, padding_ratio=0.9, path_prefix='train', include_original=False)
test_img_array = test_image.img_array.copy()

###### Preprocess & Augment Train Images

In [ ]:
# train_label_array = np.concatenate((train_label_array, train_label_array), axis=0)

In [ ]:
train_label_array.shape

In [ ]:
MAX_LENGTH = 12
DEVICE = 'mps:0'
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Create Dataset Class

In [ ]:
class BDC_Dataset(Dataset):
    
    def __init__(self, img_array, label_array, processor, max_target_length=MAX_LENGTH):
        self.img_array = img_array
        self.label_array = label_array
        self.processor = processor
        self.max_target_length = max_target_length
    
    def __len__(self):
        return len(self.img_array)

    def __getitem__(self, idx):
        # get iimage + label
        image = self.img_array[idx]
        label = self.label_array[idx]
        # prepare image (i.e. resize + normalize)
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values.to(DEVICE)
        # add labels (input_ids) by encoding the label
        labels = self.processor.tokenizer(label, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id 
                  else -100 for label in labels]
        
        encoding = {"pixel_values" : pixel_values.squeeze(), "labels" : torch.tensor(labels).to(DEVICE)}
        return encoding

##### Basic Values/Constants

In [ ]:
MODEL_CKPT = "microsoft/trocr-base-printed"
MODEL_NAME =  MODEL_CKPT.split("/")[-1] + "_bdc_license_plates_ocrV3"
NUM_OF_EPOCHS = 2
BATCH_SIZE = 2

##### Instantiate Processor, Create Training, & Testing Dataset Instances

In [ ]:
processor = TrOCRProcessor.from_pretrained(MODEL_CKPT)


test_ds = BDC_Dataset(test_img_array, test_label_array, processor=processor)

##### Print Length of Training & Testing Datasets

In [ ]:
# print(f"The training dataset has {len(train_ds)} samples in it.")
print(f"The testing dataset has {len(test_ds)} samples in it.")

##### Example of Input Data Shapes

In [ ]:
# encoding = train_ds[0]

# for k,v in encoding.items():
#     print(k, " : ", v.shape)

##### Show Example

In [ ]:
# image = Image.open(train_ds.root_dir + train_dataset['file_name'][0]).convert("RGB")

# image

##### Show Label for Above Example

In [ ]:
# labels = encoding['labels']
# labels[labels == -100] = processor.tokenizer.pad_token_id
# label_str = processor.decode(labels, skip_special_tokens=True)
# print(label_str)

#### Instantiate Model

In [ ]:
from torch.utils.data import DataLoader


eval_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained(MODEL_CKPT)


model.to(DEVICE)

##### Model Configuration Modifications

In [ ]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = MAX_LENGTH
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

##### Define Metrics Evaluation

In [ ]:
from metrics import character_accuracy_np as char_acc 

cer_metric = load_metric("cer")

def compute_metrics(pred_ids, label_ids):

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    # tmp_hat_test = [np.asarray(list(x)) for x in pred_str]
    # pred_str = np.asarray(tmp_hat_test)
    # tmp_label_test = [np.asarray(list(x)) for x in label_str]
    # label_str = np.asarray(tmp_label_test)
    # acc = char_acc(label_str, pred_str)

    return cer

In [ ]:
from tqdm import tqdm
def manual_eval(model, processor, images):
    output = []
    px_vals = []
    for image in tqdm(images):
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(DEVICE)
        px_vals.append(pixel_values)
    for pixel_values in tqdm(px_vals):
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        generated_text = generated_text.replace(" ", "") # Remove spacing
        output.append(generated_text)
    return np.asarray(output)

In [ ]:
masked_img = {'bw_image': np.array([10, 41, 23, 15, 16, 7, 28, 81, 73, 83, 59, 89, 100, 112, 148, 152, 214, 250, 292, 309, 385, 410, 434, 419, 446, 490, 464, 505, 566]),
              'gray_image': np.array([521]),
              'segment_number': np.array([2, 6, 7, 8, 9, 15, 16, 28, 29, 39, 37, 33, 42, 23, 21, 20, 80, 81, 82, 83, 74, 71, 54, 69, 59, 77, 79, 89, 99, 96, 85, 104, 105, 106, 114, 115, 116, 117, 119, 124, 125, 128, 130, 138, 139, 148, 149, 152, 153, 154, 156, 158, 159, 165, 167, 177, 180, 182, 197, 201, 202, 203, 204, 214, 216, 220, 223, 227, 235, 239, 243, 247, 248, 250, 251, 252, 258, 262, 265, 271, 273, 279, 288, 292, 294, 297, 299, 310, 307, 313, 315, 319, 320, 321, 323, 324, 330, 331, 332, 333, 335, 343, 345, 348, 350, 357, 360, 363, 364, 369, 370, 377, 381, 383, 384, 385, 389, 396, 398, 399, 402, 416, 419, 422, 426, 428, 429, 434, 440, 442, 450, 458, 459, 460, 461, 462, 464, 466, 469, 473, 478, 479, 486, 490, 505, 506, 521, 528, 529, 531, 539, 550, 551, 553, 557, 563, 566, 568, 579, 589, 585, 592, 594, 597, 595, 599, 604, 606, 608, 629, 628]),
              }
train_image = PreprocessImage(img_paths=train_path_array, output_pixel=384, padding_ratio=0.9, path_prefix='train', include_original=False)
# train_image_array = train_image.gray_image(masked_img=np.array([]))
original = train_image.img_array.copy()
original_label = train_label_array.copy()
print(f'Original image shape= {original.shape}, label= {original_label.shape}')
bw_image = train_image.bw_image(masked_img['bw_image'])
bw_image_label = train_label_array.copy()
bw_image_label = np.delete(bw_image_label, masked_img['bw_image'])
print(f'bw_image image shape= {bw_image.shape}, label= {bw_image_label.shape}')
gray_image = train_image.gray_image(masked_img['gray_image'])
gray_image_label = train_label_array.copy()
gray_image_label = np.delete(gray_image_label, masked_img['gray_image'])
print(f'gray_image image shape= {gray_image.shape}, label= {gray_image_label.shape}')
segment_number = train_image.segment_number_image(masked_img['segment_number'])
segment_number_label = train_label_array.copy()
segment_number_label = np.delete(segment_number_label, masked_img['segment_number'])
print(f'segment_number image shape= {segment_number.shape}, label= {segment_number_label.shape}')


In [ ]:
train_image_list = [bw_image, gray_image, segment_number]
train_label_list = [bw_image_label, gray_image_label, segment_number_label]

In [ ]:
from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

max_epoch = 2

for epoch in range(max_epoch):  # loop over the dataset multiple times
    
    # Generate augmented image data for each epoch
    if epoch>=0:
        augmenter = AugmentImage(image_array=original, label_array=original_label, num_augmentations=1)
        augment_original, augment_original_label = augmenter.transform()
        # new_train_image_list = train_image_list.copy()
        # new_train_label_list = train_label_list.copy()
        # for i in range(len(train_image_list)):
        #     augmenter = AugmentImage(image_array=train_image_list[i], label_array=train_label_list[i], num_augmentations=2+epoch)
        #     tmp_array, tmp_array_label = augmenter.transform()
        #     random_indices = np.random.choice(train_image_list[i].shape[0], size=train_image_list[i].shape[0]//(2*(epoch+2)), replace=False)
        #     new_train_image_list[i] = tmp_array[random_indices]
        #     new_train_label_list[i] = tmp_array_label[random_indices]
        # new_train_image_list.append(augment_original)
        # new_train_label_list.append(augment_original_label)
        # train_image_array = np.concatenate(new_train_image_list, axis=0)
        # train_lbl_array = np.concatenate(new_train_label_list, axis=0)
        # augmenter = AugmentImage(image_array=train_img_array, label_array=train_label_array, num_augmentations=2)
        # train_image_array, train_lbl_array = augmenter.transform()
    # elif epoch < 3:
    #     train_img_array = np.concatenate((train_image_list[epoch], original), axis=0)
    #     train_label_array = np.concatenate((train_label_list[epoch], original_label), axis=0)
    #     augmenter = AugmentImage(image_array=train_img_array, label_array=train_label_array, num_augmentations=2)
    #     train_image_array, train_lbl_array = augmenter.transform()
    # else:
    #     augmenter = AugmentImage(image_array=original, label_array=original_label, num_augmentations=2)
    #     train_image_array, train_lbl_array = augmenter.transform()
    train_ds = BDC_Dataset(augment_original, augment_original_label, processor=processor)
    train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    # train
    model.train()
    train_loss = 0.0
    for idx, batch in enumerate(tqdm(train_dataloader, desc=f'Epoch: {epoch+1}/{max_epoch}')):
        # get the inputs
        for k,v in batch.items():
            batch[k] = v.to(DEVICE)

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        
        if idx % (len(train_dataloader)//10) == 0 and idx != 0:
            print(f"Loss after batch {idx}:", train_loss/idx)

    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
    # evaluate
    model.eval()
    valid_cer = 0.0
    valid_acc = 0.0
    with torch.no_grad():
        yhat_test = manual_eval(model=model, processor=processor, images = test_img_array)
        tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
        yhat_test = np.asarray(tmp_hat_test)
        tmp_label_test = [np.asarray(list(x)) for x in test_label_array]
        ylabel_test = np.asarray(tmp_label_test)
        valid_acc = char_acc(ylabel_test, yhat_test)

        for batch in tqdm(eval_dataloader):
            # run batch generation
            outputs = model.generate(batch["pixel_values"].to(DEVICE))
            # compute metrics
            cer = compute_metrics(pred_ids=outputs, label_ids=batch["labels"])
            valid_cer += cer 
        

    print("Validation CER:", valid_cer / len(eval_dataloader))
    print("Validation Char_acc:", valid_acc)


In [ ]:
yhat_test


In [ ]:
model.save_pretrained(MODEL_NAME)

In [ ]:

processor.tokenizer(augment_original_label[1], padding="max_length", max_length=12).input_ids

In [ ]:
del model

In [ ]:
augment_original_label[2]

##### Model Evaluate

In [ ]:
from tqdm import tqdm
def manual_eval(model, processor, images):
    output = []
    px_vals = []
    for image in tqdm(images):
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        px_vals.append(pixel_values)
    for pixel_values in tqdm(px_vals):
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        output.append(generated_text)
    return np.asarray(output)

In [ ]:
# bdc_model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
# 

In [ ]:
from metrics import character_accuracy_np

# yhat_train = manual_eval(model=bdc_model, processor=processor, images = train_image_array)
# tmp_hat = [np.asarray(list(x)) for x in yhat_train]
# yhat_train = np.asarray(tmp_hat)
# print(f"Train dataset char_acc = {character_accuracy_np(train_label_array, yhat_train)}")

yhat_test = manual_eval(model=model, processor=processor, images = test_img_array)
tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
yhat_test = np.asarray(tmp_hat_test)
tmp_label_test = [np.asarray(list(x)) for x in test_label_array]
ylabel_test = np.asarray(tmp_label_test)
print(f"test dataset char_acc = {character_accuracy_np(ylabel_test, yhat_test)}")

In [ ]:
df_gt = pd.read_csv('ground_truth_manual.csv')
gt = df_gt['Ground Truth'].to_numpy()
gt_test_path_array = df_gt['Name of File'].to_numpy()

gt_test_image = PreprocessImage(img_paths=gt_test_path_array, output_pixel=384, padding_ratio=0.9, path_prefix='test', include_original=False)
gt_test_img_array = gt_test_image.img_array.copy()
yhat_test = manual_eval(model=model, processor=processor, images = gt_test_img_array)
tmp_hat_test = [np.asarray(list(x)) for x in yhat_test]
yhat_test = np.asarray(tmp_hat_test)
tmp_label_test = [np.asarray(list(x)) for x in gt]
ylabel_test = np.asarray(tmp_label_test)
print(f"test dataset compared to GroundTruth char_acc = {char_acc(ylabel_test, yhat_test)}")


In [ ]:
del model

### Notes & Other Takeaways From This Project
****
- The results were pretty good. I was pondering whether to train for 2 or 3 epochs. Ultimately, I trained this model for 2 epochs. If this were a work project (where multiprocessing and other options are available), I would have trained for 3, if not 4, epochs.
****

### Citations

##### For Transformer Checkpoint
- @misc{li2021trocr,
      title={TrOCR: Transformer-based Optical Character Recognition with Pre-trained Models}, 
      author={Minghao Li and Tengchao Lv and Lei Cui and Yijuan Lu and Dinei Florencio and Cha Zhang and Zhoujun Li and Furu Wei},
      year={2021},
      eprint={2109.10282},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}

##### For CER Metric
- @inproceedings{morris2004,
author = {Morris, Andrew and Maier, Viktoria and Green, Phil},
year = {2004},
month = {01},
pages = {},
title = {From WER and RIL to MER and WIL: improved evaluation measures for connected speech recognition.}
}